# from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import math
import numpy as np
import plotly as pl
import plotly.graph_objects as go
import seaborn
import pandas
import matplotlib.pyplot as plt
import sys
import os

local_path = os.path.dirname(os.path.abspath('__file__'))
outfile = os.path.join(local_path, "Abstract_Foundation_Design.py")
print(outfile)

from Abstract_Foundation_Design import Foundation
from fpdf import FPDF



In [2]:
class CSection:
    def __init__(self, width_a, depth_b, flange_length_c, thickness_t):
        self.width_a = width_a
        self.depth_b = depth_b
        self.flange_length_c = flange_length_c
        self.thickness_t = thickness_t

    def area(self):
        return ((self.width_a + 2 * self.flange_length_c) * self.thickness_t)
    
    def surface_area_per_unit_length(self):
        return (2 * self.flange_length_c + self.width_a + 2 * self.depth_b) / 1000
    
    def __str__(self):
        return f"CSection: Width A = {self.width_a}, Depth B = {self.depth_b}, Flange Length C = {self.flange_length_c}, Thickness T = {self.thickness_t}"


# --------------------------------------------------------------------------------------------------------------------------------

class CPileSectionSelector:
    
    factor_safety_set = {
                         "base": [1, 1.7, 1.5],
                         "shaft_compression": [1, 1.5, 1.3],
                         "total_compression": [1, 1.7, 1.5],
                         "shaft_tension": [1, 2, 1.7],
                         "lateral_resistance": [1, 1.5, 1.5]
                        }
    
    
    def return_correlation_factors_epsilon_BS(self, number_of_tests, a_correlation_reduction_factor):
        a = a_correlation_reduction_factor
        if number_of_tests == 1:
            return [1.55 / a, 1.55 / a]
        elif number_of_tests == 2:
            return [1.47 / a, 1.35 / a]
        elif number_of_tests == 3:
            return [1.42 / a, 1.23 / a]
        elif number_of_tests == 4:
            return [1.38 / a, 1.15 / a]
        elif number_of_tests >= 5:
            b = 1 if 1.08 / a < 1 else 1.08 / a
            return [1.35 / a, b]
    
    def return_correlation_factors_epsilon_EUROCODE(self, number_of_tests, a_correlation_reduction_factor):
        a = a_correlation_reduction_factor
        if number_of_tests == 1:
            return [1.4 / a, 1.4 / a]
        elif number_of_tests == 2:
            return [1.3 / a, 1.2 / a]
        elif number_of_tests == 3:
            return [1.2 / a, 1.05 / a]
        elif number_of_tests == 4:
            return [1.1 / a, 1.0 / a]
        elif number_of_tests >= 5:
            return [1.0/ a, 1.0 / a]
        
    
    def __init__(self, pot_data, test_type, section, a_correlation_reduction_factor, length_of_pile_Ld=2.5):
        self.a_correlation_reduction_factor = a_correlation_reduction_factor
        self.test_type = test_type
        self.c_section_data_selector = []
        self.pot_data = pot_data
        self.section = section
        self.length_of_pile_Ld = length_of_pile_Ld
        
        
    def add_c_section(self, c_section):
        self.c_section_data_selector.append(c_section)
        
# --------------------------------------------------------------------------------------------------------------------------------

    def run_calculations(self):
        
        if self.test_type == "shaft_tension":
            self.return_design_value_tension(self.a_correlation_reduction_factor)
            self.calculate_additional_sections_tension()
            self.print_results_tension()
            self.print_additional_sections_tension()
            
        elif self.test_type == "lateral_resistance":
            self.return_design_value_lateral_resistance(self.a_correlation_reduction_factor)
            self.calculate_additional_sections_lateral_resistance()
            self.print_results_lateral_resistance()
            self.print_additional_sections_lateral_resistance()
            
        elif self.test_type == "shaft_compression":
            self.return_design_value_compression(self.a_correlation_reduction_factor)
            self.calculate_additional_sections_compression()
            self.print_results_compression()
            self.print_additional_sections_compression()
            
# --------------------------------------------------------------------------------------------------------------------------------
            
    def return_design_value_tension(self, a_correlation_reduction_factor):
        
        correlation_factors_epsilon = self.return_correlation_factors_epsilon_EUROCODE(len(self.pot_data), a_correlation_reduction_factor)
        
        self.R_c_m_mean = sum(self.pot_data) / len(self.pot_data)
        self.R_c_m_min = min(self.pot_data)
        
        self.R_c_k_characteristic = self.R_c_m_mean / correlation_factors_epsilon[0] if self.R_c_m_mean / correlation_factors_epsilon[0] <= self.R_c_m_min / correlation_factors_epsilon[1] else self.R_c_m_min / correlation_factors_epsilon[1]
    
        self.R_c_design_R1 = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][0] 
        self.R_c_design_R4_without_SLS = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][1] 
        self.R_c_design_R4_with_SLS = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][2] 
        
        self.skin_surface_area = self.section.surface_area_per_unit_length() * self.length_of_pile_Ld
        
        self.q_su_R1 = self.R_c_design_R1 / self.skin_surface_area
        self.q_su_R4_without_SLS = self.R_c_design_R4_without_SLS / self.skin_surface_area
        self.q_su_R4_with_SLS = self.R_c_design_R4_with_SLS / self.skin_surface_area
    
    def return_design_value_lateral_resistance(self, a_correlation_reduction_factor):
        
        correlation_factors_epsilon = self.return_correlation_factors_epsilon_EUROCODE(len(self.pot_data), a_correlation_reduction_factor)
        
        self.R_c_m_mean = sum(self.pot_data) / len(self.pot_data)
        self.R_c_m_min = min(self.pot_data)
        
        self.R_c_k_characteristic = self.R_c_m_mean / correlation_factors_epsilon[0] if self.R_c_m_mean / correlation_factors_epsilon[0] <= self.R_c_m_min / correlation_factors_epsilon[1] else self.R_c_m_min / correlation_factors_epsilon[1]
        
        self.R_c_design_R1 = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][0] 
        self.R_c_design_R4_without_SLS = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][1] 
        self.R_c_design_R4_with_SLS = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][2] 
        
        self.q_lateral_R1 = self.R_c_design_R1 / self.section.width_a
        self.q_lateral_R4_without_SLS = self.R_c_design_R4_without_SLS / self.section.width_a
        self.q_lateral_R4_with_SLS = self.R_c_design_R4_with_SLS / self.section.width_a
        
    def return_design_value_compression(self, a_correlation_reduction_factor):

        # Determine the correlation factors using the EUROCODE method
        correlation_factors_epsilon = self.return_correlation_factors_epsilon_EUROCODE(len(self.pot_data), a_correlation_reduction_factor)

        # Calculate mean and minimum values of the POT data
        self.R_c_m_mean = sum(self.pot_data) / len(self.pot_data)
        self.R_c_m_min = min(self.pot_data)

        # Determine the characteristic resistance value based on the correlation factors
        self.R_c_k_characteristic = self.R_c_m_mean / correlation_factors_epsilon[0] if self.R_c_m_mean / correlation_factors_epsilon[0] <= self.R_c_m_min / correlation_factors_epsilon[1] else self.R_c_m_min / correlation_factors_epsilon[1]

        # Calculate the design values for different safety factors
        self.R_c_design_R1 = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][0]
        self.R_c_design_R4_without_SLS = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][1]
        self.R_c_design_R4_with_SLS = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][2]

        # Calculate the total surface area (skin surface area) of the pile
        self.skin_surface_area = self.section.surface_area_per_unit_length() * self.length_of_pile_Ld

        # Calculate the unit compressive resistance (q_cu) for different design values
        self.q_cu_R1 = self.R_c_design_R1 / self.skin_surface_area
        self.q_cu_R4_without_SLS = self.R_c_design_R4_without_SLS / self.skin_surface_area
        self.q_cu_R4_with_SLS = self.R_c_design_R4_with_SLS / self.skin_surface_area

# --------------------------------------------------------------------------------------------------------------------------------

    def calculate_additional_sections_tension(self):
        self.additional_section_results = []

        for sec in self.c_section_data_selector:
            skin_surface_area = sec.surface_area_per_unit_length() * self.length_of_pile_Ld

            R_c_R1 = self.q_su_R1 * skin_surface_area
            R_c_R4_without_SLS = self.q_su_R4_without_SLS * skin_surface_area
            R_c_R4_with_SLS = self.q_su_R4_with_SLS * skin_surface_area

            self.additional_section_results.append({
                "section": sec,
                "R_c_R1": R_c_R1,
                "R_c_R4_without_SLS": R_c_R4_without_SLS,
                "R_c_R4_with_SLS": R_c_R4_with_SLS,
            })
    
    def calculate_additional_sections_lateral_resistance(self):
        self.lateral_resistance_results = []

        for sec in self.c_section_data_selector:
            R_c_R1 = self.q_lateral_R1 * sec.width_a
            R_c_R4_without_SLS = self.q_lateral_R4_without_SLS * sec.width_a
            R_c_R4_with_SLS = self.q_lateral_R4_with_SLS * sec.width_a
            
            self.lateral_resistance_results.append({
                "section": sec,
                "R_c_R1": R_c_R1,
                "R_c_R4_without_SLS": R_c_R4_without_SLS,
                "R_c_R4_with_SLS": R_c_R4_with_SLS,
            })

            
    def calculate_additional_sections_compression(self):
        self.compression_results = []

        for sec in self.c_section_data_selector:
            skin_surface_area = sec.surface_area_per_unit_length() * self.length_of_pile_Ld

            R_c_R1 = self.q_cu_R1 * skin_surface_area
            R_c_R4_without_SLS = self.q_cu_R4_without_SLS * skin_surface_area
            R_c_R4_with_SLS = self.q_cu_R4_with_SLS * skin_surface_area

            self.compression_results.append({
                "section": sec,
                "R_c_R1": R_c_R1,
                "R_c_R4_without_SLS": R_c_R4_without_SLS,
                "R_c_R4_with_SLS": R_c_R4_with_SLS,
            })
        
# --------------------------------------------------------------------------------------------------------------------------------

    def print_additional_sections_tension(self):
        
        print("ADDITIONAL SECTIONS RESULTS")
        print(f"------------------------------")
        for index, result in enumerate(self.additional_section_results, 1):
            print(f"Additional Section {index} ")
            print(f"------------------------------")
            print("{:<80} {:>10}".format(f"R_c_R1 for Section {index}:", f"{result['R_c_R1']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_c_R4_without_SLS for Section {index}:", f"{result['R_c_R4_without_SLS']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_c_R4_with_SLS for Section {index}:", f"{result['R_c_R4_with_SLS']:.2f}kN"))
            print()

            
    def print_additional_sections_lateral_resistance(self):
        
        print("ADDITIONAL SECTIONS RESULTS")
        print(f"------------------------------")
        for index, result in enumerate(self.lateral_resistance_results, 1):
            print(f"Additional Section {index} ")
            print(f"------------------------------")
            print("{:<80} {:>10}".format(f"R_c_R1 for Section {index}:", f"{result['R_c_R1']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_c_R4_without_SLS for Section {index}:", f"{result['R_c_R4_without_SLS']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_c_R4_with_SLS for Section {index}:", f"{result['R_c_R4_with_SLS']:.2f}kN"))
            print()
            
            
    def print_additional_sections_compression(self):
        print("ADDITIONAL SECTIONS RESULTS")
        print(f"------------------------------")
        for index, result in enumerate(self.compression_results, 1):
            print(f"Additional Section {index} ")
            print(f"------------------------------")
            print("{:<80} {:>10}".format(f"R_c_R1 for Section {index}:", f"{result['R_c_R1']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_c_R4_without_SLS for Section {index}:", f"{result['R_c_R4_without_SLS']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_c_R4_with_SLS for Section {index}:", f"{result['R_c_R4_with_SLS']:.2f}kN"))
            print()

# --------------------------------------------------------------------------------------------------------------------------------

    def print_results_tension(self):

        print()
        print("PILE SECTION DESIGN RESULTS")
        print(f"------------------------------")
        print()
        
        # Geometry
        print(f"------------------------------")
        print("Geometry of CSection")
        print(f"------------------------------")

        print()
        # POT Data and Tests
        print(f"------------------------------")
        print("POT Data and Tests")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Number of POT Tests:", len(self.pot_data)))
        print("{:<80} {:>10}".format("POT Data:", f"{self.pot_data}"))

        print()
        # Design Values
        print(f"------------------------------")
        print("Design Values")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("R_c_m_mean:", f"{self.R_c_m_mean:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_m_min:", f"{self.R_c_m_min:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_k_characteristic:", f"{self.R_c_k_characteristic:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R1:", f"{self.R_c_design_R1:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R4_without_SLS:", f"{self.R_c_design_R4_without_SLS:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R4_with_SLS:", f"{self.R_c_design_R4_with_SLS:.2f}kN"))
        
    def print_results_lateral_resistance(self):
        
        print()
        print("LATERAL RESISTANCE RESULTS")
        print(f"------------------------------")
        
        print()
        
        # Print POT Data and Tests
        print(f"Number of POT Tests: {len(self.pot_data)}")
        print(f"POT Data: {self.pot_data}")
        print()
        
        # Print Design Values
        print(f"R_c_m_mean: {self.R_c_m_mean:.2f}kN")
        print(f"R_c_m_min: {self.R_c_m_min:.2f}kN")
        print(f"R_c_k_characteristic: {self.R_c_k_characteristic:.2f}kN")
        print()
    
    
    def print_results_compression(self):

        print()
        print("PILE SECTION COMPRESSION RESULTS")
        print(f"------------------------------")
        print()

        # Geometry
        print(f"------------------------------")
        print("Geometry of CSection")
        print(f"------------------------------")

        print()
        # POT Data and Tests
        print(f"------------------------------")
        print("POT Data and Tests")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Number of POT Tests:", len(self.pot_data)))
        print("{:<80} {:>10}".format("POT Data:", f"{self.pot_data}"))

        print()
        # Design Values
        print(f"------------------------------")
        print("Design Values")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("R_c_m_mean:", f"{self.R_c_m_mean:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_m_min:", f"{self.R_c_m_min:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_k_characteristic:", f"{self.R_c_k_characteristic:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R1:", f"{self.R_c_design_R1:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R4_without_SLS:", f"{self.R_c_design_R4_without_SLS:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R4_with_SLS:", f"{self.R_c_design_R4_with_SLS:.2f}kN"))
        print()
        
        
# --------------------------------------------------------------------------------------------------------------------------------

    def generate_pdf_tension(self, pdf, area):
        pdf.add_page()
        pdf.set_margins(left=10, top=10, right=10)
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(190, 10, f"PILE SECTION TENSION RESISTANCE RESULTS for {area}", align='C')
        pdf.multi_cell(190, 10, "------------------------------", align='C')
        pdf.multi_cell(0, 10, '')


        # Geometry
#         pdf.multi_cell(190, 10, "Geometry of CSection", align='C')

#         for index, c_section in enumerate(self.c_section_data_selector, 1):
#             pdf.multi_cell(190, 10, f"CSection {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")

        # POT Data and Tests
        pdf.multi_cell(190, 10, "POT Data and Tests", align='C')
        pdf.multi_cell(190, 10, f"Number of POT Tests: {len(self.pot_data)}")
        pdf.multi_cell(190, 10, f"POT Data: {self.pot_data}")
        pdf.multi_cell(0, 10, '')


        # Design Values
        pdf.multi_cell(190, 10, "Design Values", align='C')
        pdf.multi_cell(190, 10, f"R_c_m_mean: {self.R_c_m_mean:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_m_min: {self.R_c_m_min:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_k_characteristic: {self.R_c_k_characteristic:.2f}kN")
        pdf.multi_cell(0, 10, '')


        pdf.multi_cell(190, 10, f"R_c_design_R1: {self.R_c_design_R1:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_design_R4_without_SLS: {self.R_c_design_R4_without_SLS:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_design_R4_with_SLS: {self.R_c_design_R4_with_SLS:.2f}kN")
        pdf.multi_cell(0, 10, '')


        # Additional Sections
        pdf.multi_cell(190, 10, "ADDITIONAL SECTIONS RESULTS", align='C')
        pdf.multi_cell(0, 10, '')


        for index, result in enumerate(self.additional_section_results, 1):
            c_section = self.c_section_data_selector[index - 1]
            
            pdf.multi_cell(190, 10, f"C Section {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")
            pdf.multi_cell(190, 10, f"R_c_R1 for Section {index}: {result['R_c_R1']:.2f}kN")
            pdf.multi_cell(190, 10, f"R_c_R4_without_SLS for Section {index}: {result['R_c_R4_without_SLS']:.2f}kN")
            pdf.multi_cell(190, 10, f"R_c_R4_with_SLS for Section {index}: {result['R_c_R4_with_SLS']:.2f}kN")
            pdf.multi_cell(0, 10, '')


    def generate_pdf_lateral_resistance(self, pdf, area):
        pdf.add_page()
        pdf.set_margins(left=10, top=10, right=10)
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(190, 10, f"PILE SECTION LATERAL RESISTANCE RESULTS for {area}", align='C')
        pdf.multi_cell(190, 10, "------------------------------", align='C')
        pdf.multi_cell(0, 10, '')


        # Geometry
#         pdf.multi_cell(190, 10, "Geometry of CSection", align='C')
#         for index, c_section in enumerate(self.c_section_data_selector, 1):
#             pdf.multi_cell(190, 10, f"CSection {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")

        # POT Data and Tests
        pdf.multi_cell(190, 10, "POT Data and Tests", align='C')
        pdf.multi_cell(190, 10, f"Number of POT Tests: {len(self.pot_data)}")
        pdf.multi_cell(190, 10, f"POT Data: {self.pot_data}")
        pdf.multi_cell(0, 10, '')


        # Design Values
        pdf.multi_cell(190, 10, "Design Values", align='C')
        pdf.multi_cell(190, 10, f"R_c_m_mean: {self.R_c_m_mean:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_m_min: {self.R_c_m_min:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_k_characteristic: {self.R_c_k_characteristic:.2f}kN")
        pdf.multi_cell(0, 10, '')


        pdf.multi_cell(190, 10, f"R_c_design_R1: {self.R_c_design_R1:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_design_R4_without_SLS: {self.R_c_design_R4_without_SLS:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_design_R4_with_SLS: {self.R_c_design_R4_with_SLS:.2f}kN")
        pdf.multi_cell(0, 10, '')


        # Lateral Resistance
        pdf.multi_cell(190, 10, "Lateral Resistance Results", align='C')
        pdf.multi_cell(0, 10, '')


        for index, result in enumerate(self.lateral_resistance_results, 1):
            c_section = self.c_section_data_selector[index - 1]
            
            pdf.multi_cell(190, 10, f"C Section {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")
            pdf.multi_cell(190, 10, f"R_c_R1 for Section {index}: {result['R_c_R1']:.2f}kN")
            pdf.multi_cell(190, 10, f"R_c_R4_without_SLS for Section {index}: {result['R_c_R4_without_SLS']:.2f}kN")
            pdf.multi_cell(190, 10, f"R_c_R4_with_SLS for Section {index}: {result['R_c_R4_with_SLS']:.2f}kN")
            pdf.multi_cell(0, 10, '')

    def generate_pdf_compression(self, pdf, area):
        pdf.add_page()
        pdf.set_margins(left=10, top=10, right=10)
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(190, 10, f"PILE SECTION COMPRESSION RESISTANCE RESULTS for {area}", align='C')
        pdf.multi_cell(190, 10, "------------------------------", align='C')
        pdf.multi_cell(0, 10, '')

        # Geometry (Optional, you can add this section if needed)
        # pdf.multi_cell(190, 10, "Geometry of CSection", align='C')
        # for index, c_section in enumerate(self.c_section_data_selector, 1):
        #     pdf.multi_cell(190, 10, f"CSection {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")

        # POT Data and Tests
        pdf.multi_cell(190, 10, "POT Data and Tests", align='C')
        pdf.multi_cell(190, 10, f"Number of POT Tests: {len(self.pot_data)}")
        pdf.multi_cell(190, 10, f"POT Data: {self.pot_data}")
        pdf.multi_cell(0, 10, '')

        # Design Values
        pdf.multi_cell(190, 10, "Design Values", align='C')
        pdf.multi_cell(190, 10, f"R_c_m_mean: {self.R_c_m_mean:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_m_min: {self.R_c_m_min:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_k_characteristic: {self.R_c_k_characteristic:.2f}kN")
        pdf.multi_cell(0, 10, '')

        pdf.multi_cell(190, 10, f"R_c_design_R1: {self.R_c_design_R1:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_design_R4_without_SLS: {self.R_c_design_R4_without_SLS:.2f}kN")
        pdf.multi_cell(190, 10, f"R_c_design_R4_with_SLS: {self.R_c_design_R4_with_SLS:.2f}kN")
        pdf.multi_cell(0, 10, '')

        # Additional Sections Results
        pdf.multi_cell(190, 10, "ADDITIONAL SECTIONS RESULTS", align='C')
        pdf.multi_cell(0, 10, '')

        for index, result in enumerate(self.compression_results, 1):
            c_section = self.c_section_data_selector[index - 1]

            pdf.multi_cell(190, 10, f"C Section {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")
            pdf.multi_cell(190, 10, f"R_c_R1 for Section {index}: {result['R_c_R1']:.2f}kN")
            pdf.multi_cell(190, 10, f"R_c_R4_without_SLS for Section {index}: {result['R_c_R4_without_SLS']:.2f}kN")
            pdf.multi_cell(190, 10, f"R_c_R4_with_SLS for Section {index}: {result['R_c_R4_with_SLS']:.2f}kN")
            pdf.multi_cell(0, 10, '')

# --------------------------------------------------------------------------------------------------------------------------------
        


In [3]:
# THINGS TO CHANGE HERE!!!!!

section1 = CSection(250, 110, 55, 12)
section2 = CSection(200, 100, 50, 10)
section3 = CSection(180, 90, 45, 9)
section4 = CSection(160, 80, 35, 7)
section5 = CSection(140, 60, 30, 5)

section6 = CSection(120, 70, 14, 3)

section7 = CSection(100, 50, 15, 3)

#data = {"ZONE 1": [15, 20, 25, 25, 25], "ZONE 2": [25, 15, 12.5, 12, 15], "ZONE 3": [25, 20, 25, 25, 15], "ZONE 4": [20, 16, 12.5, 20, 15]}

#data2 = {"ZONE 1": [5, 5, 5, 5, 5], "ZONE 2": [5, 5, 5, 5, 5], "ZONE 3": [5, 5, 5, 5, 7.5], "ZONE 4": [7.5, 7.5, 5, 5, 7.5]}

data = {"ZONE 1": [4.7088, 5, 4.8069, 3.5316, 29.43, 23.544, 6.03315, 8.7309, 6.42555]}

data_compression = {'ZONE 1': [20.601, 20.601, 20.601, 20.601, 20.601, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544]}

data_tension_1_8 = {'ZONE 1': [4.8069, 5.0031, 6.03315, 6.42555, 8.7309]}
data_tension_2 = {'ZONE 1': [5.15025, 5.6898, 11.5758, 22.6611, 23.544]}

data_lateral = {"ZONE 1": [18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 18.639, 8.829, 8.829, 7.848]}

data_tension_1_8_single = {'ZONE 1': [10.246]}
data_tension_2_single = {'ZONE 1': [13.320]}

a_correlation_reduction_factor = 1.1


pdf = FPDF()
pdf.set_auto_page_break(auto=1, margin=15)

for area, pot_data in data_compression.items():
    selector = CPileSectionSelector(pot_data, "shaft_compression", section6, a_correlation_reduction_factor, length_of_pile_Ld=2)
    
    # Add your c_sections here (You would replace these lines with your actual sections)
    selector.add_c_section(section6)

    selector.run_calculations()
    selector.generate_pdf_compression(pdf, area)

# for area, pot_data in data2.items():
#     selector = CPileSectionSelector(pot_data, "lateral_resistance", section7, a_correlation_reduction_factor, length_of_pile_Ld=2.5)
    
#     # Add your c_sections here (You would replace these lines with your actual sections)
#     selector.add_c_section(section1)
#     selector.add_c_section(section2)
#     selector.add_c_section(section3)
#     selector.add_c_section(section4)
#     selector.add_c_section(section5)
#     selector.add_c_section(section6)
#     selector.add_c_section(section7)

#     selector.run_calculations()
#     selector.generate_pdf_lateral_resistance(pdf, area)
    

pdf.output(name="ROMSEY_POT_Results.pdf", dest="F").encode('latin1')


PILE SECTION COMPRESSION RESULTS
------------------------------

------------------------------
Geometry of CSection
------------------------------

------------------------------
POT Data and Tests
------------------------------
Number of POT Tests:                                                                     20
POT Data:                                                                        [20.601, 20.601, 20.601, 20.601, 20.601, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544, 23.544]

------------------------------
Design Values
------------------------------
R_c_m_mean:                                                                         22.81kN
R_c_m_min:                                                                          20.60kN
R_c_k_characteristic:                                                               22.66kN
R_c_design_R1:                                                                    

b''